<a href="https://colab.research.google.com/gist/maclandrol/03ac6c1f4cbb02f923787c628c201921/09_TorchXRayVision_Pathologie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Enseignant:** Emmanuel Noutahi, PhD

# Tutorial 4 : Détection et Localisation de Pathologies
# Tutorial 4: Pathology Detection and Localization

---

## 📚 Contexte Médical / Medical Context

### Pour les Étudiants en Médecine / For Medical Students
La **détection automatique de pathologies** représente une application cruciale de l'IA en radiologie :
- **Anatomie pathologique** : Reconnaissance des anomalies structurelles
- **Physiologie** : Compréhension de l'impact fonctionnel des lésions
- **Radiologie** : Aide au diagnostic et à la localisation précise

### Pour les Praticiens / For Practitioners
- **Chirurgiens** : Localisation pré-opératoire des lésions et planification
- **Médecins Généralistes** : Aide au dépistage et triage des urgences
- **Enseignants** : Outil pédagogique pour l'apprentissage des pathologies

---

## 🎯 Objectifs d'Apprentissage / Learning Objectives

À la fin de ce tutoriel, vous serez capable de :
1. Utiliser les modèles TorchXRayVision pour détecter 18 pathologies courantes
2. Localiser les zones pathologiques sur l'image
3. Interpréter les scores de probabilité et seuils de décision
4. Générer des cartes de chaleur (heatmaps) pour la localisation
5. Créer un rapport diagnostique automatique

---

## 🏥 Pathologies Détectables / Detectable Pathologies

TorchXRayVision peut détecter **18 pathologies** principales :

### Pathologies Pulmonaires / Lung Pathologies
- **Atelectasis** (Atélectasie) - Collapsus pulmonaire
- **Consolidation** - Condensation parenchymateuse
- **Infiltration** - Infiltrats pulmonaires
- **Pneumothorax** - Air dans l'espace pleural
- **Edema** (Œdème) - Œdème pulmonaire
- **Emphysema** (Emphysème) - Destruction alvéolaire
- **Fibrosis** (Fibrose) - Fibrose pulmonaire
- **Pleural_Thickening** - Épaississement pleural

### Pathologies Cardiaques / Cardiac Pathologies
- **Cardiomegaly** (Cardiomégalie) - Augmentation cardiaque

### Autres Pathologies / Other Pathologies
- **Nodule** - Nodules pulmonaires
- **Mass** (Masse) - Masses thoraciques
- **Pneumonia** (Pneumonie) - Infection pulmonaire
- **Hernia** (Hernie) - Hernie diaphragmatique
- **Lung Lesion** - Lésions pulmonaires
- **Fracture** - Fractures costales
- **Lung Opacity** - Opacités pulmonaires
- **Enlarged Cardiomediastinum** - Élargissement médiastinal
- **Support Devices** - Dispositifs médicaux

---

## 📋 Prérequis / Prerequisites

Ce tutoriel fait suite aux **Tutoriels 1, 2 et 3**. Vous devriez maîtriser :
- Chargement et préparation d'images
- Classification avec TorchXRayVision
- Segmentation anatomique

---

## 🔧 Installation et Configuration / Setup

In [ ]:
# Installation des bibliothèques nécessaires / Install required libraries
!pip install torchxrayvision
!pip install torch torchvision
!pip install matplotlib seaborn
!pip install numpy pandas
!pip install scikit-image opencv-python
!pip install grad-cam  # Pour les cartes d'activation

print("✅ Installation terminée / Installation completed")

In [ ]:
# Import des bibliothèques / Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchxrayvision as xrv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from PIL import Image
import cv2
from google.colab import files
import io
import warnings
import pickle
from datetime import datetime
warnings.filterwarnings('ignore')

# Configuration de l'affichage / Display configuration
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print("📚 Bibliothèques importées avec succès / Libraries imported successfully")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🏥 TorchXRayVision version: {xrv.__version__}")

# Vérification du GPU / GPU check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"💻 Device utilisé / Device used: {device}")

# Liste des pathologies détectables
PATHOLOGIES = [
    'Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 
    'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule',
    'Mass', 'Pneumonia', 'Hernia', 'Lung Lesion', 'Fracture', 'Lung Opacity',
    'Enlarged Cardiomediastinum', 'Support Devices'
]

print(f"🏥 Pathologies détectables: {len(PATHOLOGIES)}")

## 🤖 Chargement des Modèles de Détection / Loading Detection Models

In [ ]:
# Chargement des modèles spécialisés / Load specialized models
print("🔄 Chargement des modèles de détection de pathologies...")
print("🔄 Loading pathology detection models...")

# 1. Modèle DenseNet pour classification générale
try:
    model_densenet = xrv.models.DenseNet(weights="densenet121-res224-all")
    model_densenet.to(device)
    model_densenet.eval()
    print("✅ DenseNet121 chargé / loaded")
except Exception as e:
    print(f"❌ Erreur DenseNet: {e}")
    model_densenet = None

# 2. Modèle CheXpert pour pathologies spécifiques
try:
    model_chexpert = xrv.models.DenseNet(weights="densenet121-res224-chexpert")
    model_chexpert.to(device)
    model_chexpert.eval()
    print("✅ CheXpert DenseNet chargé / loaded")
except Exception as e:
    print(f"❌ Erreur CheXpert: {e}")
    model_chexpert = None

# 3. Modèle NIH pour comparaison
try:
    model_nih = xrv.models.DenseNet(weights="densenet121-res224-nih")
    model_nih.to(device)
    model_nih.eval()
    print("✅ NIH DenseNet chargé / loaded")
except Exception as e:
    print(f"❌ Erreur NIH: {e}")
    model_nih = None

# Sélectionner le modèle principal
if model_densenet is not None:
    main_model = model_densenet
    model_name = "DenseNet121-All"
elif model_chexpert is not None:
    main_model = model_chexpert
    model_name = "CheXpert"
elif model_nih is not None:
    main_model = model_nih
    model_name = "NIH"
else:
    raise Exception("Aucun modèle n'a pu être chargé")

print(f"\n🎯 Modèle principal: {model_name}")
print(f"📊 Nombre de classes détectées: {len(main_model.pathologies)}")
print(f"📋 Classes disponibles: {main_model.pathologies}")

## 📤 Chargement d'Image / Image Upload

### Option 1 : Charger depuis le Tutorial 3

In [ ]:
# Tentative de chargement des résultats du Tutorial 3
try:
    with open('segmentation_results_tutorial3.pkl', 'rb') as f:
        segmentation_data = pickle.load(f)
    
    original_image = segmentation_data['original_image']
    lung_mask = segmentation_data['lung_mask']
    cardiac_mask = segmentation_data['cardiac_mask']
    
    print("✅ Données du Tutorial 3 chargées / Tutorial 3 data loaded")
    print(f"📏 Taille de l'image / Image size: {original_image.shape}")
    
    use_tutorial3_data = True
    
except FileNotFoundError:
    print("ℹ️ Données du Tutorial 3 non trouvées, création d'une nouvelle image")
    print("ℹ️ Tutorial 3 data not found, creating new image")
    use_tutorial3_data = False
    
    # Créer une image d'exemple avec pathologies simulées
    img_array = np.random.rand(224, 224) * 0.3 + 0.4
    
    # Simuler des structures normales
    img_array[50:180, 30:100] *= 0.7  # Poumon gauche
    img_array[50:180, 124:194] *= 0.7  # Poumon droit
    img_array[120:180, 90:134] *= 1.3  # Cœur
    
    # Simuler quelques pathologies
    # Nodule dans le poumon droit
    img_array[80:90, 150:160] *= 1.8
    
    # Infiltration dans le poumon gauche
    img_array[100:130, 60:80] *= 1.4
    
    # Légère cardiomégalie
    img_array[110:190, 85:139] *= 1.2
    
    original_image = img_array
    lung_mask = None
    cardiac_mask = None

# Affichage de l'image
plt.figure(figsize=(8, 8))
plt.imshow(original_image, cmap='gray')
plt.title("Image pour Détection de Pathologies\nImage for Pathology Detection")
plt.axis('off')
plt.show()

### Option 2 : Charger une Nouvelle Image / Upload New Image

In [ ]:
# Option pour charger une nouvelle image
print("📤 Voulez-vous charger une nouvelle image avec des pathologies ?")
print("📤 Would you like to upload a new image with pathologies?")
print("")
print("Cliquez 'Choisir des fichiers' pour charger une radiographie avec pathologies")
print("Click 'Choose Files' to upload an X-ray with pathologies")

# Interface de chargement
uploaded = files.upload()

# Traitement de l'image chargée
if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"📁 Nouveau fichier chargé / New file uploaded: {filename}")
    
    # Charger et traiter l'image
    uploaded_image = Image.open(io.BytesIO(uploaded[filename]))
    
    # Convertir en niveaux de gris
    if uploaded_image.mode != 'L':
        uploaded_image = uploaded_image.convert('L')
    
    # Redimensionner et normaliser
    uploaded_array = np.array(uploaded_image)
    if uploaded_array.shape != (224, 224):
        uploaded_array = cv2.resize(uploaded_array, (224, 224))
    
    original_image = uploaded_array.astype(np.float32) / 255.0
    lung_mask = None  # Reset des masques pour la nouvelle image
    cardiac_mask = None
    
    print("✅ Nouvelle image préparée / New image prepared")
    
    # Affichage de la nouvelle image
    plt.figure(figsize=(8, 8))
    plt.imshow(original_image, cmap='gray')
    plt.title(f"Image Chargée: {filename}")
    plt.axis('off')
    plt.show()
else:
    print("ℹ️ Utilisation de l'image précédente / Using previous image")

## 🔧 Préparation pour Détection / Preprocessing for Detection

In [ ]:
def preprocess_for_pathology_detection(image):
    """
    Préparation spécifique pour la détection de pathologies
    Specific preprocessing for pathology detection
    """
    print("🔧 Préparation pour la détection de pathologies...")
    print("🔧 Preprocessing for pathology detection...")
    
    # Si l'image est déjà un array numpy
    if isinstance(image, np.ndarray):
        img_array = image.copy()
    else:
        img_array = np.array(image)
    
    # S'assurer que l'image est en 224x224
    if img_array.shape != (224, 224):
        img_array = cv2.resize(img_array, (224, 224))
    
    # Normalisation spécifique à TorchXRayVision
    if img_array.max() > 1:
        img_array = img_array.astype(np.float32) / 255.0
    
    # Normalisation Z-score
    mean = np.mean(img_array)
    std = np.std(img_array)
    img_normalized = (img_array - mean) / (std + 1e-8)
    
    # Convertir en tensor PyTorch avec la forme attendue [1, 1, 224, 224]
    img_tensor = torch.FloatTensor(img_normalized)
    img_tensor = img_tensor.unsqueeze(0).unsqueeze(0)  # Ajouter dimensions batch et canal
    img_tensor = img_tensor.to(device)
    
    print(f"📏 Forme finale du tensor / Final tensor shape: {img_tensor.shape}")
    print(f"📊 Valeurs min/max / Min/max values: {img_tensor.min():.3f} / {img_tensor.max():.3f}")
    
    return img_tensor, img_normalized

# Préparer l'image pour la détection
img_tensor, img_preprocessed = preprocess_for_pathology_detection(original_image)

print("✅ Image préparée pour la détection / Image prepared for detection")

## 🔍 Détection de Pathologies / Pathology Detection

### Analyse avec Modèle Principal / Analysis with Main Model

In [ ]:
def detect_pathologies(model, img_tensor, threshold=0.5):
    """
    Détection des pathologies avec le modèle
    Pathology detection with the model
    """
    print("🔍 Détection des pathologies en cours...")
    print("🔍 Pathology detection in progress...")
    
    # Inférence avec le modèle
    with torch.no_grad():
        outputs = model(img_tensor)
        
        # Appliquer la fonction sigmoïde pour obtenir des probabilités
        probabilities = torch.sigmoid(outputs)
        probabilities_np = probabilities.cpu().numpy().flatten()
    
    # Créer un DataFrame avec les résultats
    pathologies_list = model.pathologies
    
    results_df = pd.DataFrame({
        'Pathologie': pathologies_list,
        'Probabilité': probabilities_np,
        'Détectée': probabilities_np > threshold
    })
    
    # Trier par probabilité décroissante
    results_df = results_df.sort_values('Probabilité', ascending=False).reset_index(drop=True)
    
    print(f"📊 Analyse terminée pour {len(pathologies_list)} pathologies")
    print(f"🎯 Seuil de détection: {threshold}")
    
    detected_count = sum(probabilities_np > threshold)
    print(f"🔴 Pathologies détectées: {detected_count}/{len(pathologies_list)}")
    
    return results_df, probabilities_np, outputs

# Effectuer la détection
results, probabilities, raw_outputs = detect_pathologies(main_model, img_tensor, threshold=0.3)

# Afficher les résultats
print("\n" + "="*60)
print("🏥 RÉSULTATS DE DÉTECTION DE PATHOLOGIES")
print("🏥 PATHOLOGY DETECTION RESULTS")
print("="*60)

# Top 10 des pathologies les plus probables
print("\n🔝 TOP 10 - PATHOLOGIES LES PLUS PROBABLES:")
for i, row in results.head(10).iterrows():
    status = "🔴 DÉTECTÉE" if row['Détectée'] else "🟢 Non détectée"
    print(f"   {i+1:2d}. {row['Pathologie']:20s} : {row['Probabilité']:.3f} ({row['Probabilité']*100:.1f}%) - {status}")

# Pathologies détectées (seuil > 0.3)
detected_pathologies = results[results['Détectée']]
if len(detected_pathologies) > 0:
    print(f"\n🚨 PATHOLOGIES DÉTECTÉES (Probabilité > 30%):")
    for i, row in detected_pathologies.iterrows():
        confidence = "ÉLEVÉE" if row['Probabilité'] > 0.7 else "MOYENNE" if row['Probabilité'] > 0.5 else "FAIBLE"
        print(f"   • {row['Pathologie']:20s} : {row['Probabilité']:.3f} ({row['Probabilité']*100:.1f}%) - Confiance {confidence}")
else:
    print("\n✅ AUCUNE PATHOLOGIE DÉTECTÉE avec le seuil actuel")

print("\n" + "="*60)

## 📊 Visualisation des Résultats / Results Visualization

In [ ]:
def visualize_pathology_results(image, results_df, probabilities):
    """
    Visualisation complète des résultats de détection
    Comprehensive visualization of detection results
    """
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('ANALYSE COMPLÈTE DE DÉTECTION DE PATHOLOGIES\nCOMPREHENSIVE PATHOLOGY DETECTION ANALYSIS', 
                fontsize=16, fontweight='bold')
    
    # 1. Image originale
    axes[0, 0].imshow(image, cmap='gray')
    axes[0, 0].set_title('Image Analysée\nAnalyzed Image', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    
    # 2. Graphique en barres des probabilités (Top 10)
    top_10 = results_df.head(10)
    colors = ['red' if detected else 'lightblue' for detected in top_10['Détectée']]
    
    bars = axes[0, 1].barh(range(len(top_10)), top_10['Probabilité'], color=colors, alpha=0.7)
    axes[0, 1].set_yticks(range(len(top_10)))
    axes[0, 1].set_yticklabels([p[:15] for p in top_10['Pathologie']], fontsize=10)
    axes[0, 1].set_xlabel('Probabilité')
    axes[0, 1].set_title('Top 10 Pathologies\n(Rouge = Détectée)', fontsize=12, fontweight='bold')
    axes[0, 1].axvline(x=0.3, color='orange', linestyle='--', alpha=0.7, label='Seuil (0.3)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for i, (bar, prob) in enumerate(zip(bars, top_10['Probabilité'])):
        width = bar.get_width()
        axes[0, 1].text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                       f'{prob:.3f}', ha='left', va='center', fontweight='bold')
    
    # 3. Distribution des probabilités
    axes[0, 2].hist(probabilities, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 2].axvline(x=0.3, color='orange', linestyle='--', linewidth=2, label='Seuil (0.3)')
    axes[0, 2].axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Seuil (0.5)')
    axes[0, 2].set_xlabel('Probabilité')
    axes[0, 2].set_ylabel('Nombre de Pathologies')
    axes[0, 2].set_title('Distribution des\nProbabilités', fontsize=12, fontweight='bold')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Pathologies détectées (seulement si il y en a)
    detected = results_df[results_df['Détectée']]
    if len(detected) > 0:
        # Graphique en secteurs des pathologies détectées
        sizes = detected['Probabilité']
        labels = [f"{path[:12]}\n{prob:.3f}" for path, prob in zip(detected['Pathologie'], detected['Probabilité'])]
        colors_pie = plt.cm.Reds(np.linspace(0.4, 0.9, len(detected)))
        
        wedges, texts, autotexts = axes[1, 0].pie(sizes, labels=labels, colors=colors_pie, 
                                                 autopct='%1.1f%%', startangle=90)
        axes[1, 0].set_title(f'Pathologies Détectées\n({len(detected)} trouvées)', 
                           fontsize=12, fontweight='bold')
    else:
        axes[1, 0].text(0.5, 0.5, 'AUCUNE\nPATHOLOGIE\nDÉTECTÉE', 
                       ha='center', va='center', transform=axes[1, 0].transAxes,
                       fontsize=14, fontweight='bold', color='green')
        axes[1, 0].set_title('Statut de Détection', fontsize=12, fontweight='bold')
    
    # 5. Comparaison par catégories de pathologies
    # Catégoriser les pathologies
    lung_pathologies = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 
                       'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Pneumonia',
                       'Lung Lesion', 'Lung Opacity', 'Nodule', 'Mass']
    cardiac_pathologies = ['Cardiomegaly', 'Enlarged Cardiomediastinum']
    other_pathologies = ['Hernia', 'Fracture', 'Support Devices']
    
    # Calculer moyennes par catégorie
    lung_probs = [prob for path, prob in zip(results_df['Pathologie'], results_df['Probabilité']) 
                  if path in lung_pathologies]
    cardiac_probs = [prob for path, prob in zip(results_df['Pathologie'], results_df['Probabilité']) 
                     if path in cardiac_pathologies]
    other_probs = [prob for path, prob in zip(results_df['Pathologie'], results_df['Probabilité']) 
                   if path in other_pathologies]
    
    categories = []
    avg_probs = []
    
    if lung_probs:
        categories.append('Pulmonaires')
        avg_probs.append(np.mean(lung_probs))
    if cardiac_probs:
        categories.append('Cardiaques')
        avg_probs.append(np.mean(cardiac_probs))
    if other_probs:
        categories.append('Autres')
        avg_probs.append(np.mean(other_probs))
    
    if categories:
        bars_cat = axes[1, 1].bar(categories, avg_probs, 
                                 color=['lightcoral', 'lightblue', 'lightgreen'][:len(categories)], 
                                 alpha=0.7, edgecolor='black')
        axes[1, 1].set_ylabel('Probabilité Moyenne')
        axes[1, 1].set_title('Analyse par Catégorie\nde Pathologies', fontsize=12, fontweight='bold')
        axes[1, 1].set_ylim(0, max(avg_probs) * 1.2 if avg_probs else 1)
        
        # Ajouter les valeurs sur les barres
        for bar, prob in zip(bars_cat, avg_probs):
            height = bar.get_height()
            axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                           f'{prob:.3f}', ha='center', va='bottom', fontweight='bold')
        
        axes[1, 1].grid(True, alpha=0.3)
    
    # 6. Matrice de confiance/seuils
    thresholds = np.arange(0.1, 0.9, 0.1)
    detection_counts = []
    
    for thresh in thresholds:
        count = sum(probabilities > thresh)
        detection_counts.append(count)
    
    axes[1, 2].plot(thresholds, detection_counts, 'bo-', linewidth=2, markersize=8)
    axes[1, 2].set_xlabel('Seuil de Probabilité')
    axes[1, 2].set_ylabel('Nombre de Détections')
    axes[1, 2].set_title('Sensibilité au Seuil\nde Détection', fontsize=12, fontweight='bold')
    axes[1, 2].grid(True, alpha=0.3)
    
    # Marquer le seuil actuel
    current_count = sum(probabilities > 0.3)
    axes[1, 2].scatter([0.3], [current_count], color='red', s=100, zorder=5)
    axes[1, 2].annotate(f'Seuil actuel\n({current_count} détections)', 
                       xy=(0.3, current_count), xytext=(0.5, current_count + 2),
                       arrowprops=dict(arrowstyle='->', color='red'))
    
    plt.tight_layout()
    plt.show()

# Créer la visualisation
visualize_pathology_results(img_preprocessed, results, probabilities)

print("✅ Visualisation des résultats générée / Results visualization generated")

## 🔥 Cartes de Chaleur pour Localisation / Heatmaps for Localization

Les **cartes de chaleur** (heatmaps) permettent de visualiser où le modèle "regarde" pour détecter les pathologies.

In [ ]:
def generate_grad_cam_heatmap(model, img_tensor, target_class_idx):
    """
    Génération de cartes de chaleur Grad-CAM pour localisation
    Generate Grad-CAM heatmaps for localization
    """
    model.eval()
    
    # Hook pour capturer les gradients
    gradients = []
    activations = []
    
    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])
    
    def forward_hook(module, input, output):
        activations.append(output)
    
    # Trouver la dernière couche de convolution
    target_layer = None
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            target_layer = module
    
    if target_layer is None:
        print("❌ Aucune couche de convolution trouvée")
        return None
    
    # Enregistrer les hooks
    h1 = target_layer.register_forward_hook(forward_hook)
    h2 = target_layer.register_backward_hook(backward_hook)
    
    # Forward pass
    img_tensor.requires_grad_()
    output = model(img_tensor)
    
    # Backward pass pour la classe cible
    model.zero_grad()
    class_output = output[0, target_class_idx]
    class_output.backward()
    
    # Supprimer les hooks
    h1.remove()
    h2.remove()
    
    if gradients and activations:
        # Calculer la carte de chaleur
        grad = gradients[0].cpu().data.numpy()[0]
        activation = activations[0].cpu().data.numpy()[0]
        
        # Moyenne des gradients pour chaque canal
        weights = np.mean(grad, axis=(1, 2))
        
        # Combinaison pondérée des cartes d'activation
        heatmap = np.zeros(activation.shape[1:])
        for i, weight in enumerate(weights):
            heatmap += weight * activation[i]
        
        # ReLU et normalisation
        heatmap = np.maximum(heatmap, 0)
        if heatmap.max() > 0:
            heatmap = heatmap / heatmap.max()
        
        # Redimensionner à la taille de l'image
        heatmap_resized = cv2.resize(heatmap, (224, 224))
        
        return heatmap_resized
    
    return None

def create_heatmap_visualization(image, results_df, model, img_tensor):
    """
    Création de visualisations avec cartes de chaleur pour les pathologies détectées
    Create heatmap visualizations for detected pathologies
    """
    print("🔥 Génération des cartes de chaleur...")
    print("🔥 Generating heatmaps...")
    
    # Sélectionner les pathologies les plus probables
    top_pathologies = results_df.head(6)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('CARTES DE CHALEUR - LOCALISATION DES PATHOLOGIES\nHEATMAPS - PATHOLOGY LOCALIZATION', 
                fontsize=16, fontweight='bold')
    
    axes = axes.flatten()
    
    for idx, (_, row) in enumerate(top_pathologies.iterrows()):
        pathology = row['Pathologie']
        probability = row['Probabilité']
        detected = row['Détectée']
        
        # Trouver l'index de la pathologie dans le modèle
        if pathology in model.pathologies:
            class_idx = model.pathologies.index(pathology)
            
            try:
                # Générer la carte de chaleur
                heatmap = generate_grad_cam_heatmap(model, img_tensor, class_idx)
                
                if heatmap is not None:
                    # Afficher l'image avec superposition de la carte de chaleur
                    axes[idx].imshow(image, cmap='gray', alpha=0.7)
                    axes[idx].imshow(heatmap, cmap='jet', alpha=0.4)
                    
                    # Titre avec informations
                    status_color = 'red' if detected else 'green'
                    status_text = 'DÉTECTÉE' if detected else 'Non détectée'
                    axes[idx].set_title(f'{pathology}\nProb: {probability:.3f} - {status_text}', 
                                       fontsize=10, fontweight='bold', color=status_color)
                    
                    # Ajouter une barre de couleur
                    if probability > 0.3:  # Seulement pour les pathologies significatives
                        axes[idx].contour(heatmap, levels=[0.5, 0.7], colors=['yellow', 'red'], 
                                        linewidths=[1, 2], alpha=0.8)
                
                else:
                    # Si la carte de chaleur n'a pas pu être générée
                    axes[idx].imshow(image, cmap='gray')
                    axes[idx].set_title(f'{pathology}\nProb: {probability:.3f}\n(Carte non générée)', 
                                       fontsize=10, fontweight='bold')
            
            except Exception as e:
                print(f"⚠️ Erreur génération heatmap pour {pathology}: {e}")
                axes[idx].imshow(image, cmap='gray')
                axes[idx].set_title(f'{pathology}\nProb: {probability:.3f}\n(Erreur génération)', 
                                   fontsize=10, fontweight='bold')
        
        else:
            # Pathologie non trouvée dans le modèle
            axes[idx].imshow(image, cmap='gray')
            axes[idx].set_title(f'{pathology}\nProb: {probability:.3f}\n(Non supportée)', 
                               fontsize=10, fontweight='bold')
        
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Cartes de chaleur générées / Heatmaps generated")

# Méthode alternative simplifiée pour la localisation
def simple_localization_heatmaps(image, results_df):
    """
    Méthode alternative pour montrer les zones d'attention
    Alternative method to show attention areas
    """
    print("🎯 Génération de cartes de localisation simplifiées...")
    print("🎯 Generating simplified localization maps...")
    
    # Sélectionner les pathologies les plus probables
    top_pathologies = results_df[results_df['Probabilité'] > 0.2].head(4)
    
    if len(top_pathologies) == 0:
        print("ℹ️ Aucune pathologie avec probabilité > 0.2 pour la localisation")
        return
    
    fig, axes = plt.subplots(1, len(top_pathologies), figsize=(5*len(top_pathologies), 5))
    
    if len(top_pathologies) == 1:
        axes = [axes]
    
    for idx, (_, row) in enumerate(top_pathologies.iterrows()):
        pathology = row['Pathologie']
        probability = row['Probabilité']
        
        # Créer une carte de chaleur simulée basée sur des zones anatomiques connues
        heatmap = np.zeros((224, 224))
        
        # Zones d'attention basées sur la pathologie
        if 'Lung' in pathology or pathology in ['Atelectasis', 'Consolidation', 'Infiltration', 
                                                'Pneumothorax', 'Edema', 'Emphysema', 
                                                'Fibrosis', 'Pneumonia', 'Nodule']:
            # Zone pulmonaire
            heatmap[50:180, 30:100] = probability * 0.8  # Poumon gauche
            heatmap[50:180, 124:194] = probability * 0.8  # Poumon droit
        
        elif pathology in ['Cardiomegaly', 'Enlarged Cardiomediastinum']:
            # Zone cardiaque
            heatmap[120:180, 90:134] = probability
        
        elif pathology in ['Fracture']:
            # Zone costale
            heatmap[60:160, 20:40] = probability * 0.6  # Côtes gauches
            heatmap[60:160, 184:204] = probability * 0.6  # Côtes droites
        
        else:
            # Zone générale thoracique
            heatmap[50:180, 50:174] = probability * 0.5
        
        # Appliquer un flou gaussien pour un aspect plus réaliste
        heatmap = cv2.GaussianBlur(heatmap, (15, 15), 0)
        
        # Affichage
        axes[idx].imshow(image, cmap='gray', alpha=0.8)
        im = axes[idx].imshow(heatmap, cmap='hot', alpha=0.5, vmin=0, vmax=probability)
        
        # Titre et formatage
        status = 'DÉTECTÉE' if row['Détectée'] else 'Suspectée'
        color = 'red' if row['Détectée'] else 'orange'
        axes[idx].set_title(f'{pathology}\n{probability:.3f} ({probability*100:.1f}%)\n{status}', 
                           fontsize=12, fontweight='bold', color=color)
        axes[idx].axis('off')
        
        # Barre de couleur
        cbar = plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)
        cbar.set_label('Intensité de Détection', rotation=270, labelpad=15)
    
    plt.suptitle('LOCALISATION APPROXIMATIVE DES PATHOLOGIES\nAPPROXIMATE PATHOLOGY LOCALIZATION', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Essayer la génération de cartes de chaleur avancées
try:
    create_heatmap_visualization(img_preprocessed, results, main_model, img_tensor)
except Exception as e:
    print(f"⚠️ Cartes de chaleur avancées non disponibles: {e}")
    print("🎯 Utilisation de la méthode de localisation simplifiée...")
    
    # Utiliser la méthode simplifiée
    simple_localization_heatmaps(img_preprocessed, results)

## 📋 Rapport Clinique Automatique / Automatic Clinical Report

In [ ]:
def generate_comprehensive_clinical_report(results_df, model_name, threshold=0.3):
    """
    Génération d'un rapport clinique complet pour la détection de pathologies
    Generate comprehensive clinical report for pathology detection
    """
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    print("\n" + "="*80)
    print("🏥 RAPPORT CLINIQUE DE DÉTECTION RADIOLOGIQUE")
    print("🏥 RADIOLOGICAL DETECTION CLINICAL REPORT")
    print("="*80)
    
    print(f"📅 Date et heure d'analyse: {current_time}")
    print(f"🤖 Modèle utilisé: {model_name}")
    print(f"🎯 Seuil de détection: {threshold}")
    print(f"📊 Nombre total de pathologies analysées: {len(results_df)}")
    
    # Résumé exécutif
    detected_pathologies = results_df[results_df['Détectée']]
    high_confidence = results_df[results_df['Probabilité'] > 0.7]
    medium_confidence = results_df[(results_df['Probabilité'] > 0.5) & (results_df['Probabilité'] <= 0.7)]
    low_confidence = results_df[(results_df['Probabilité'] > threshold) & (results_df['Probabilité'] <= 0.5)]
    
    print("\n" + "-"*50)
    print("📊 RÉSUMÉ EXÉCUTIF / EXECUTIVE SUMMARY")
    print("-"*50)
    
    if len(detected_pathologies) == 0:
        print("✅ CONCLUSION PRINCIPALE: EXAMEN NORMAL")
        print("   • Aucune pathologie détectée avec le seuil de confiance actuel")
        print("   • Image compatible avec un aspect radiologique normal")
    else:
        print(f"🔴 CONCLUSION PRINCIPALE: {len(detected_pathologies)} PATHOLOGIE(S) DÉTECTÉE(S)")
        print(f"   • Confiance élevée (>70%): {len(high_confidence)} pathologie(s)")
        print(f"   • Confiance moyenne (50-70%): {len(medium_confidence)} pathologie(s)")
        print(f"   • Confiance faible ({threshold*100:.0f}-50%): {len(low_confidence)} pathologie(s)")
    
    # Détail des pathologies détectées
    if len(detected_pathologies) > 0:
        print("\n" + "-"*50)
        print("🚨 PATHOLOGIES DÉTECTÉES - DÉTAIL")
        print("-"*50)
        
        for i, row in detected_pathologies.iterrows():
            pathology = row['Pathologie']
            probability = row['Probabilité']
            
            # Niveau de confiance
            if probability > 0.7:
                confidence_level = "🔴 ÉLEVÉE"
                urgency = "Nécessite une évaluation clinique prioritaire"
            elif probability > 0.5:
                confidence_level = "🟡 MOYENNE"
                urgency = "Corrélation clinique recommandée"
            else:
                confidence_level = "🟠 FAIBLE"
                urgency = "Surveillance ou examens complémentaires"
            
            print(f"\n   {i+1}. {pathology.upper()}")
            print(f"      • Probabilité: {probability:.3f} ({probability*100:.1f}%)")
            print(f"      • Niveau de confiance: {confidence_level}")
            print(f"      • Recommandation: {urgency}")
            
            # Informations cliniques spécifiques
            clinical_info = get_clinical_information(pathology)
            if clinical_info:
                print(f"      • Contexte clinique: {clinical_info}")
    
    # Recommandations par spécialité
    print("\n" + "-"*50)
    print("👥 RECOMMANDATIONS PAR SPÉCIALITÉ")
    print("-"*50)
    
    # Pour les médecins généralistes
    print("\n👨‍⚕️ POUR LES MÉDECINS GÉNÉRALISTES:")
    if len(detected_pathologies) == 0:
        print("   ✅ Pas d'action urgente requise")
        print("   • Suivi clinique de routine")
        print("   • Surveillance des symptômes")
    else:
        urgent_findings = results_df[results_df['Probabilité'] > 0.7]
        if len(urgent_findings) > 0:
            print("   🚨 ACTIONS URGENTES REQUISES:")
            for _, row in urgent_findings.iterrows():
                print(f"      • {row['Pathologie']}: Consultation spécialisée urgente")
                if 'Pneumothorax' in row['Pathologie']:
                    print("        → URGENCE: Décompression immédiate si symptômes")
                elif 'Cardiomegaly' in row['Pathologie']:
                    print("        → Échocardiographie et consultation cardiologique")
                elif 'Mass' in row['Pathologie'] or 'Nodule' in row['Pathologie']:
                    print("        → TDM thoracique et consultation oncologique")
        
        moderate_findings = results_df[(results_df['Probabilité'] > 0.3) & (results_df['Probabilité'] <= 0.7)]
        if len(moderate_findings) > 0:
            print("   📋 SURVEILLANCE ET ÉVALUATIONS:")
            for _, row in moderate_findings.iterrows():
                print(f"      • {row['Pathologie']}: Corrélation clinique et suivi")
    
    # Pour les chirurgiens
    print("\n🏥 POUR LES CHIRURGIENS:")
    surgical_relevant = results_df[results_df['Pathologie'].isin([
        'Pneumothorax', 'Mass', 'Nodule', 'Fracture', 'Hernia'
    ]) & results_df['Détectée']]
    
    if len(surgical_relevant) > 0:
        print("   🔪 ÉVALUATIONS CHIRURGICALES NÉCESSAIRES:")
        for _, row in surgical_relevant.iterrows():
            print(f"      • {row['Pathologie']} (Prob: {row['Probabilité']:.3f})")
            if 'Pneumothorax' in row['Pathologie']:
                print("        → Considérer drainage thoracique")
            elif 'Mass' in row['Pathologie'] or 'Nodule' in row['Pathologie']:
                print("        → Évaluation pour biopsie/résection")
            elif 'Fracture' in row['Pathologie']:
                print("        → Évaluation orthopédique")
    else:
        print("   ✅ Pas d'indication chirurgicale immédiate détectée")
    
    # Pour les enseignants
    print("\n📚 POUR LES ENSEIGNANTS D'ANATOMIE/RADIOLOGIE:")
    print("   🎓 POINTS PÉDAGOGIQUES:")
    print(f"      • Analyse de {len(results_df)} pathologies différentes")
    print(f"      • Sensibilité du modèle au seuil de détection")
    print(f"      • Corrélation entre probabilités IA et signes radiologiques")
    
    if len(detected_pathologies) > 0:
        print("   📖 CAS D'ÉTUDE:")
        for _, row in detected_pathologies.head(3).iterrows():
            print(f"      • {row['Pathologie']}: Exemple d'apprentissage de reconnaissance")
    
    # Limitations et considérations
    print("\n" + "-"*50)
    print("⚠️ LIMITATIONS ET CONSIDÉRATIONS IMPORTANTES")
    print("-"*50)
    print("   • Ce rapport est généré automatiquement par IA")
    print("   • TOUJOURS corréler avec l'examen clinique du patient")
    print("   • Les probabilités ne remplacent pas le jugement médical")
    print("   • Faux positifs et faux négatifs possibles")
    print("   • Validation par radiologue recommandée pour cas complexes")
    
    # Métriques techniques
    print("\n" + "-"*50)
    print("🔬 MÉTRIQUES TECHNIQUES")
    print("-"*50)
    print(f"   • Probabilité moyenne: {results_df['Probabilité'].mean():.3f}")
    print(f"   • Probabilité maximale: {results_df['Probabilité'].max():.3f}")
    print(f"   • Écart-type des probabilités: {results_df['Probabilité'].std():.3f}")
    print(f"   • Pathologies > 10%: {len(results_df[results_df['Probabilité'] > 0.1])}")
    print(f"   • Pathologies > 50%: {len(results_df[results_df['Probabilité'] > 0.5])}")
    
    print("\n" + "="*80)
    print("📋 Rapport généré automatiquement par TorchXRayVision")
    print("📋 Report automatically generated by TorchXRayVision")
    print(f"⏰ {current_time}")
    print("="*80)

def get_clinical_information(pathology):
    """
    Informations cliniques pour chaque pathologie
    Clinical information for each pathology
    """
    clinical_info = {
        'Atelectasis': 'Collapsus alvéolaire, souvent post-opératoire',
        'Consolidation': 'Remplissage alvéolaire, évoque une pneumonie',
        'Infiltration': 'Processus inflammatoire ou infectieux diffus',
        'Pneumothorax': 'Urgence potentielle, décompression si symptomatique',
        'Edema': 'Œdème pulmonaire, évaluer fonction cardiaque',
        'Emphysema': 'BPCO, évaluation de la fonction respiratoire',
        'Fibrosis': 'Fibrose pulmonaire, bilan étiologique nécessaire',
        'Pleural_Thickening': 'Épaississement pleural, rechercher cause',
        'Cardiomegaly': 'Cardiomégalie, échocardiographie recommandée',
        'Nodule': 'Nodule pulmonaire, surveillance ou biopsie selon taille',
        'Mass': 'Masse thoracique, investigation oncologique urgente',
        'Pneumonia': 'Pneumonie, antibiothérapie et surveillance',
        'Hernia': 'Hernie diaphragmatique, évaluation chirurgicale',
        'Fracture': 'Fracture costale, analgésie et surveillance complications'
    }
    
    return clinical_info.get(pathology, 'Consulter la littérature médicale')

# Générer le rapport clinique complet
generate_comprehensive_clinical_report(results, model_name, threshold=0.3)

## 🔄 Comparaison Multi-Modèles / Multi-Model Comparison

Comparons les résultats de différents modèles TorchXRayVision :

In [ ]:
def compare_multiple_models(img_tensor):
    """
    Comparaison des résultats entre différents modèles
    Comparison of results between different models
    """
    print("🔄 Comparaison multi-modèles en cours...")
    print("🔄 Multi-model comparison in progress...")
    
    models_to_compare = []
    model_names = []
    
    # Ajouter les modèles disponibles
    if model_densenet is not None:
        models_to_compare.append(model_densenet)
        model_names.append("DenseNet-All")
    
    if model_chexpert is not None:
        models_to_compare.append(model_chexpert)
        model_names.append("CheXpert")
    
    if model_nih is not None:
        models_to_compare.append(model_nih)
        model_names.append("NIH")
    
    if len(models_to_compare) < 2:
        print("⚠️ Moins de 2 modèles disponibles pour la comparaison")
        return
    
    # Collecter les résultats de tous les modèles
    all_results = {}
    common_pathologies = None
    
    for model, name in zip(models_to_compare, model_names):
        print(f"   📊 Analyse avec {name}...")
        
        with torch.no_grad():
            outputs = model(img_tensor)
            probabilities = torch.sigmoid(outputs).cpu().numpy().flatten()
        
        # Créer un dictionnaire pathologie -> probabilité
        model_results = {}
        for pathology, prob in zip(model.pathologies, probabilities):
            model_results[pathology] = prob
        
        all_results[name] = model_results
        
        # Trouver les pathologies communes
        if common_pathologies is None:
            common_pathologies = set(model.pathologies)
        else:
            common_pathologies = common_pathologies.intersection(set(model.pathologies))
    
    # Créer une visualisation comparative
    common_pathologies = sorted(list(common_pathologies))
    
    if len(common_pathologies) == 0:
        print("⚠️ Aucune pathologie commune entre les modèles")
        return
    
    # Préparer les données pour la visualisation
    comparison_data = []
    
    for pathology in common_pathologies[:15]:  # Top 15 pour la lisibilité
        for model_name in model_names:
            if pathology in all_results[model_name]:
                comparison_data.append({
                    'Pathologie': pathology,
                    'Modèle': model_name,
                    'Probabilité': all_results[model_name][pathology]
                })
    
    df_comparison = pd.DataFrame(comparison_data)
    
    # Visualisation
    fig, axes = plt.subplots(2, 2, figsize=(20, 14))
    fig.suptitle('COMPARAISON MULTI-MODÈLES\nMULTI-MODEL COMPARISON', 
                fontsize=16, fontweight='bold')
    
    # 1. Heatmap comparative
    pivot_data = df_comparison.pivot(index='Pathologie', columns='Modèle', values='Probabilité')
    sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='Reds', ax=axes[0, 0], cbar_kws={'label': 'Probabilité'})
    axes[0, 0].set_title('Matrice Comparative des Probabilités', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Modèle')
    axes[0, 0].set_ylabel('Pathologie')
    
    # 2. Graphique en barres groupées pour le top 8
    top_pathologies = df_comparison.groupby('Pathologie')['Probabilité'].max().nlargest(8).index
    df_top = df_comparison[df_comparison['Pathologie'].isin(top_pathologies)]
    
    pathology_positions = {path: i for i, path in enumerate(top_pathologies)}
    x_positions = []
    heights = []
    colors = []
    labels = []
    
    bar_width = 0.25
    model_colors = ['skyblue', 'lightcoral', 'lightgreen', 'orange'][:len(model_names)]
    
    for i, model_name in enumerate(model_names):
        model_data = df_top[df_top['Modèle'] == model_name]
        for _, row in model_data.iterrows():
            path_pos = pathology_positions[row['Pathologie']]
            x_positions.append(path_pos + i * bar_width)
            heights.append(row['Probabilité'])
            colors.append(model_colors[i])
            if i == 0:  # Ajouter le label seulement une fois par modèle
                labels.append(model_name)
    
    for i, model_name in enumerate(model_names):
        model_data = df_top[df_top['Modèle'] == model_name]
        x_vals = [pathology_positions[row['Pathologie']] + i * bar_width for _, row in model_data.iterrows()]
        y_vals = [row['Probabilité'] for _, row in model_data.iterrows()]
        axes[0, 1].bar(x_vals, y_vals, bar_width, label=model_name, color=model_colors[i], alpha=0.7)
    
    axes[0, 1].set_xlabel('Pathologies')
    axes[0, 1].set_ylabel('Probabilité')
    axes[0, 1].set_title('Top 8 Pathologies - Comparaison par Modèle', fontsize=12, fontweight='bold')
    axes[0, 1].set_xticks([i + bar_width for i in range(len(top_pathologies))])
    axes[0, 1].set_xticklabels([p[:12] for p in top_pathologies], rotation=45, ha='right')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Corrélation entre modèles
    if len(model_names) >= 2:
        model1_data = [all_results[model_names[0]][path] for path in common_pathologies 
                      if path in all_results[model_names[0]]]
        model2_data = [all_results[model_names[1]][path] for path in common_pathologies 
                      if path in all_results[model_names[1]]]
        
        axes[1, 0].scatter(model1_data, model2_data, alpha=0.6, s=60)
        axes[1, 0].plot([0, 1], [0, 1], 'r--', alpha=0.8, label='Corrélation parfaite')
        axes[1, 0].set_xlabel(f'Probabilités {model_names[0]}')
        axes[1, 0].set_ylabel(f'Probabilités {model_names[1]}')
        axes[1, 0].set_title(f'Corrélation {model_names[0]} vs {model_names[1]}', 
                           fontsize=12, fontweight='bold')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Calculer et afficher le coefficient de corrélation
        correlation = np.corrcoef(model1_data, model2_data)[0, 1]
        axes[1, 0].text(0.05, 0.95, f'Corrélation: {correlation:.3f}', 
                       transform=axes[1, 0].transAxes, fontsize=12, 
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 4. Tableau de consensus
    axes[1, 1].axis('off')
    
    # Trouver les pathologies avec consensus (accord entre modèles)
    consensus_data = []
    for pathology in common_pathologies:
        probs = [all_results[name][pathology] for name in model_names if pathology in all_results[name]]
        if len(probs) >= 2:
            avg_prob = np.mean(probs)
            std_prob = np.std(probs)
            consensus = "Élevé" if std_prob < 0.1 else "Moyen" if std_prob < 0.2 else "Faible"
            
            if avg_prob > 0.2:  # Seulement les pathologies avec probabilité significative
                consensus_data.append([pathology[:20], f"{avg_prob:.3f}", f"{std_prob:.3f}", consensus])
    
    # Trier par probabilité moyenne décroissante et prendre le top 10
    consensus_data = sorted(consensus_data, key=lambda x: float(x[1]), reverse=True)[:10]
    
    if consensus_data:
        headers = ['Pathologie', 'Moy.', 'Écart', 'Consensus']
        table = axes[1, 1].table(cellText=consensus_data, colLabels=headers, 
                               cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 2)
        
        # Styliser l'en-tête
        for i in range(4):
            table[(0, i)].set_facecolor('#4CAF50')
            table[(0, i)].set_text_props(weight='bold', color='white')
        
        axes[1, 1].set_title('Consensus Multi-Modèles (Top 10)', 
                           fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Comparaison terminée entre {len(models_to_compare)} modèles")
    print(f"📊 {len(common_pathologies)} pathologies communes analysées")
    
    return df_comparison

# Effectuer la comparaison si plusieurs modèles sont disponibles
try:
    comparison_results = compare_multiple_models(img_tensor)
except Exception as e:
    print(f"⚠️ Comparaison multi-modèles non disponible: {e}")
    print("ℹ️ Un seul modèle disponible pour l'analyse")

## 💾 Sauvegarde des Résultats / Save Results

In [ ]:
# Sauvegarder tous les résultats du Tutorial 4
pathology_results = {
    'original_image': img_preprocessed,
    'detection_results': results,
    'probabilities': probabilities,
    'model_used': model_name,
    'threshold': 0.3,
    'detected_pathologies': results[results['Détectée']],
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

# Ajouter les résultats de segmentation si disponibles
if use_tutorial3_data:
    pathology_results['lung_mask'] = lung_mask
    pathology_results['cardiac_mask'] = cardiac_mask

# Sauvegarder en format pickle
with open('pathology_results_tutorial4.pkl', 'wb') as f:
    pickle.dump(pathology_results, f)

print("💾 Résultats sauvegardés dans 'pathology_results_tutorial4.pkl'")
print("💾 Results saved to 'pathology_results_tutorial4.pkl'")

# Créer un résumé visuel final
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Image originale
axes[0].imshow(img_preprocessed, cmap='gray')
axes[0].set_title('Image Analysée\nAnalyzed Image')
axes[0].axis('off')

# Top pathologies détectées
top_detected = results[results['Détectée']].head(5)
if len(top_detected) > 0:
    y_pos = np.arange(len(top_detected))
    axes[1].barh(y_pos, top_detected['Probabilité'], color='red', alpha=0.7)
    axes[1].set_yticks(y_pos)
    axes[1].set_yticklabels([p[:15] for p in top_detected['Pathologie']])
    axes[1].set_xlabel('Probabilité')
    axes[1].set_title(f'Pathologies Détectées\n({len(top_detected)} trouvées)')
    axes[1].grid(True, alpha=0.3)
else:
    axes[1].text(0.5, 0.5, 'AUCUNE\nPATHOLOGIE\nDÉTECTÉE', 
                ha='center', va='center', transform=axes[1].transAxes,
                fontsize=16, fontweight='bold', color='green')
    axes[1].set_title('Statut de Détection')

# Distribution générale des probabilités
axes[2].hist(probabilities, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[2].axvline(x=0.3, color='red', linestyle='--', linewidth=2, label='Seuil (0.3)')
axes[2].set_xlabel('Probabilité')
axes[2].set_ylabel('Nombre de Pathologies')
axes[2].set_title('Distribution des\nProbabilités')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.suptitle('RÉSUMÉ DU TUTORIAL 4 - DÉTECTION DE PATHOLOGIES\nTUTORIAL 4 SUMMARY - PATHOLOGY DETECTION', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('pathology_detection_summary_tutorial4.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*70)
print("🎉 TUTORIAL 4 TERMINÉ AVEC SUCCÈS!")
print("🎉 TUTORIAL 4 COMPLETED SUCCESSFULLY!")
print("="*70)

print(f"\n📊 RÉSUMÉ DES ACCOMPLISSEMENTS:")
print(f"   ✅ Analyse de {len(results)} pathologies")
print(f"   ✅ Détection de {len(results[results['Détectée']])} pathologies avec seuil 0.3")
print(f"   ✅ Génération de cartes de localisation")
print(f"   ✅ Rapport clinique complet généré")
print(f"   ✅ Recommandations par spécialité médicale")

if use_tutorial3_data:
    print(f"   ✅ Intégration avec les résultats de segmentation du Tutorial 3")

print(f"\n🎯 COMPÉTENCES ACQUISES:")
print(f"   • Détection automatique de pathologies thoraciques")
print(f"   • Interprétation des scores de probabilité")
print(f"   • Localisation approximative des anomalies")
print(f"   • Génération de rapports cliniques automatiques")
print(f"   • Recommandations par spécialité médicale")

print(f"\n🚀 PROCHAINES ÉTAPES:")
print(f"   📚 Tutorial 5: Multi-Model Comparison (Comparaison approfondie)")
print(f"   📚 Tutorial 6: Custom Dataset Integration (Données personnalisées)")
print(f"   📚 Applications cliniques avancées")

print("\n" + "="*70)